In [9]:
import tensorflow as tf
import pandas as pd

from stable_baselines.common.policies import MlpLnLstmPolicy
from stable_baselines import PPO2

from tensortrade.strategies import StableBaselinesTradingStrategy
from tensortrade.environments import TradingEnvironment
from tensortrade.rewards import RiskAdjustedReturns
from tensortrade.actions import ManagedRiskOrders
from tensortrade.instruments import Quantity, TradingPair, BTC, USD
from tensortrade.orders.criteria import StopLoss, StopDirection
from tensortrade.wallets import Wallet, Portfolio
from tensortrade.exchanges.simulated import SimulatedExchange
from tensortrade.features.stationarity import LogDifference
from tensortrade.features.scalers import MinMaxNormalizer
from tensortrade.features import FeaturePipeline

WINDOW_SIZE = 1
PRICE_COLUMN = 'close'

normalize = MinMaxNormalizer(inplace=True)
difference = LogDifference(inplace=True)
feature_pipeline = FeaturePipeline(steps=[normalize])

action_scheme = ManagedRiskOrders(pairs=[USD/BTC])
reward_scheme = RiskAdjustedReturns(return_algorithm="sortino")

#csv_file = tf.keras.utils.get_file(
#    'Coinbase_BTCUSD_1h.csv', 'https://www.cryptodatadownload.com/cdd/Coinbase_BTCUSD_1h.csv')
csv_file='data/Coinbase_BTCUSD_1h.csv'
ohlcv_data = pd.read_csv(csv_file, skiprows=1, index_col="date")
ohlcv_data.columns = map(str.lower, ohlcv_data.columns)
ohlcv_data = ohlcv_data.rename(columns={'volume btc': 'volume'})
df = pd.read_csv('data/Coinbase_BTCUSD_1h.csv', skiprows=1)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d %I-%p')
df = df.set_index('date').sort_index()

exchange = SimulatedExchange(data_frame=df, price_column=PRICE_COLUMN, randomize_time_slices=True)


In [10]:
exchange.data_frame

,symbol,open,high,low,close,volume_btc,volume
date,,,,,,,
2017-07-01 11:00:00,BTCUSD,2505.56,2513.38,2495.12,2509.17,114.60,287000.32
2017-07-01 12:00:00,BTCUSD,2509.17,2512.87,2484.99,2488.43,157.36,393142.50
2017-07-01 13:00:00,BTCUSD,2488.43,2488.43,2454.40,2454.43,280.28,693254.01
2017-07-01 14:00:00,BTCUSD,2454.43,2473.93,2450.83,2459.35,289.42,712864.80
2017-07-01 15:00:00,BTCUSD,2459.35,2475.00,2450.00,2467.83,276.82,682105.41
2017-07-01 16:00:00,BTCUSD,2467.83,2469.99,2450.00,2460.01,330.01,811494.29
2017-07-01 17:00:00,BTCUSD,2460.01,2460.68,2428.03,2445.99,370.26,904722.49
2017-07-01 18:00:00,BTCUSD,2445.99,2455.86,2414.00,2425.94,536.06,1303397.79
2017-07-01 19:00:00,BTCUSD,2425.94,2437.86,2406.44,2435.99,461.20,1118954.38


In [ ]:
wallets = [(exchange, USD, 10000), (exchange, BTC, 0)]

portfolio = Portfolio(base_instrument=USD, wallets=wallets)

environment = TradingEnvironment(exchange=exchange,
                                 portfolio=portfolio,
                                 action_scheme=action_scheme,
                                 reward_scheme=reward_scheme,
                                 feature_pipeline=feature_pipeline,
                                 window_size=WINDOW_SIZE,
                                 observe_wallets=[USD, BTC])

print('Observation Data:')
print(environment.observation_columns)

model = PPO2
policy = MlpLnLstmPolicy
params = { "learning_rate": 1e-5, 'nminibatches': 1 }

strategy = StableBaselinesTradingStrategy(environment=environment,
                                          model=model,
                                          policy=policy,
                                          model_kwargs=params)

performance = strategy.run(steps=10000)

Observation Data:
['open' 'high' 'low' 'close' 'volume_btc' 'volume' 'USD' 'BTC'
 'USD_pending' 'BTC_pending']
Execute:  <Order: id=8e638818-466b-460a-b535-b1847002eb6c, status=0, type=market, side=buy, pair=USD/BTC, quantity=3000.00 USD, size=3000.0, price=2488.43, criteria=None, path_id=8e638818-466b-460a-b535-b1847002eb6c | Recipes: <Recipe: id=b84da7ca-1f7c-46cf-8f52-a9d0c18547c1, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L)>>
Fill:  <Trade: id=f7f17a94-9ad0-4c17-8fb0-d7eec35c8c6e, order_id=8e638818-466b-460a-b535-b1847002eb6c, step=1, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=2991.00 USD, price=2488.43, commission=9.00 USD>
Total traded:  3000.0
Completed:  8e638818-466b-460a-b535-b1847002eb6c <Order: id=3a7c398c-e7f2-40fa-af3f-0806fa4b501c, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.20196268 BTC, size=2991.0, price=2488.43, criteria=StopLoss: either | 0.03 (T/P) | 0.06 (S/L), path_id=8e638818-466b-460a-b53

Execute:  <Order: id=6e8137a0-382f-4d7c-9dca-b6be37b49685, status=0, type=market, side=buy, pair=USD/BTC, quantity=0.08 USD, size=0.08, price=2391.87, criteria=None, path_id=6e8137a0-382f-4d7c-9dca-b6be37b49685 | Recipes: <Recipe: id=56797670-ee28-444e-bf4f-2ce74dadcef6, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.02 (T/P) | 0.06 (S/L)>>
Fill:  <Trade: id=9b7e1202-e013-40a2-b324-38a92ce0973c, order_id=6e8137a0-382f-4d7c-9dca-b6be37b49685, step=13, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=0.08 USD, price=2391.87, commission=0.00 USD>
Total traded:  0.08
Completed:  6e8137a0-382f-4d7c-9dca-b6be37b49685 <Order: id=116b9b6c-02d4-41ca-b813-50e018fd18f6, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.00003345 BTC, size=0.08, price=2391.87, criteria=StopLoss: either | 0.02 (T/P) | 0.06 (S/L), path_id=6e8137a0-382f-4d7c-9dca-b6be37b49685 | Recipes: >
Execute:  <Order: id=a3c0542c-fc33-49b0-8662-3898d3df7114, status=0, type=market, side=buy, pa

Execute:  <Order: id=f40fdf5e-b469-4775-a8b9-1c24dd2ec0fb, status=0, type=market, side=buy, pair=USD/BTC, quantity=3.92 USD, size=3.92, price=2517.01, criteria=None, path_id=f40fdf5e-b469-4775-a8b9-1c24dd2ec0fb | Recipes: <Recipe: id=be004a90-848e-406c-9980-9bb5dd10d118, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.01 (T/P) | 0.02 (S/L)>>
Fill:  <Trade: id=b531be85-9dab-4eb0-ae56-d82c19e179b3, order_id=f40fdf5e-b469-4775-a8b9-1c24dd2ec0fb, step=35, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=3.91 USD, price=2517.01, commission=0.01 USD>
Total traded:  3.92
Completed:  f40fdf5e-b469-4775-a8b9-1c24dd2ec0fb <Order: id=a059c408-88a0-4bc1-ba62-8344295bd6d3, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.00155343 BTC, size=3.91, price=2517.01, criteria=StopLoss: either | 0.01 (T/P) | 0.02 (S/L), path_id=f40fdf5e-b469-4775-a8b9-1c24dd2ec0fb | Recipes: >
Execute:  <Order: id=85a3793b-d4a5-46ae-ba00-6ac92585dd88, status=0, type=market, side=buy, pa

Execute:  <Order: id=ebdf4d77-a138-4066-9be5-aebbd5b0c53f, status=0, type=market, side=buy, pair=USD/BTC, quantity=185.88 USD, size=185.88, price=2547.43, criteria=None, path_id=ebdf4d77-a138-4066-9be5-aebbd5b0c53f | Recipes: <Recipe: id=3b772a1c-2576-4e39-b626-43a71ddc882c, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.03 (T/P) | 0.04 (S/L)>>
Fill:  <Trade: id=2bd538c2-69e6-4d5c-9134-977df5c66f13, order_id=ebdf4d77-a138-4066-9be5-aebbd5b0c53f, step=56, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=185.32 USD, price=2547.43, commission=0.56 USD>
Total traded:  185.88
Completed:  ebdf4d77-a138-4066-9be5-aebbd5b0c53f <Order: id=69d03dae-0761-41b2-b856-f98bbdb25338, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.07274783 BTC, size=185.32, price=2547.43, criteria=StopLoss: either | 0.03 (T/P) | 0.04 (S/L), path_id=ebdf4d77-a138-4066-9be5-aebbd5b0c53f | Recipes: >
Execute:  <Order: id=a4e5da71-75e6-4bd3-b997-e8ef7a106293, status=0, type=market, si

Execute:  <Order: id=cec5d06a-560b-4969-b1b5-d9c9941e7035, status=0, type=market, side=buy, pair=USD/BTC, quantity=158.20 USD, size=158.2, price=2594.49, criteria=None, path_id=cec5d06a-560b-4969-b1b5-d9c9941e7035 | Recipes: <Recipe: id=737c6c51-641b-43e5-b774-971c113e6112, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.02 (T/P) | 0.02 (S/L)>>
Fill:  <Trade: id=72693223-1118-469a-b300-51ca2a3e1f8d, order_id=cec5d06a-560b-4969-b1b5-d9c9941e7035, step=69, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=157.73 USD, price=2594.49, commission=0.47 USD>
Total traded:  158.2
Completed:  cec5d06a-560b-4969-b1b5-d9c9941e7035 <Order: id=aab7d9d2-1edf-481e-b3f1-2261e3dc339d, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.06079422 BTC, size=157.73, price=2594.49, criteria=StopLoss: either | 0.02 (T/P) | 0.02 (S/L), path_id=cec5d06a-560b-4969-b1b5-d9c9941e7035 | Recipes: >
Execute:  <Order: id=eb843c5f-da9c-44aa-8fee-fe26486ef7cc, status=0, type=market, side

...

Execute:  <Order: id=e7d73d72-0de3-4358-9dfc-7c27f59df0f2, status=0, type=market, side=buy, pair=USD/BTC, quantity=2816.45 USD, size=2816.45, price=2177.91, criteria=None, path_id=e7d73d72-0de3-4358-9dfc-7c27f59df0f2 | Recipes: <Recipe: id=1fdcc732-e773-41df-95ed-e1914a0b6425, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.01 (T/P) | 0.02 (S/L)>>
Fill:  <Trade: id=4346051d-a533-4e39-a3cc-89ec6a8ba278, order_id=e7d73d72-0de3-4358-9dfc-7c27f59df0f2, step=322, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=2808.00 USD, price=2177.91, commission=8.45 USD>
Total traded:  2816.45
Completed:  e7d73d72-0de3-4358-9dfc-7c27f59df0f2 <Order: id=c6fd75d3-b120-4f1e-be14-1a67f1bb07cd, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.28930948 BTC, size=2808.0, price=2177.91, criteria=StopLoss: either | 0.01 (T/P) | 0.02 (S/L), path_id=e7d73d72-0de3-4358-9dfc-7c27f59df0f2 | Recipes: >
Execute:  <Order: id=c6fd75d3-b120-4f1e-be14-1a67f1bb07cd, status=0, type=marke

Completed:  34f6b9ce-6472-4e2a-9d25-003749df8035 <Order: id=534b628f-66d2-4248-8784-08d80a4e154b, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.17757892 BTC, size=372.09, price=2095.35, criteria=StopLoss: either | 0.02 (T/P) | 0.02 (S/L), path_id=34f6b9ce-6472-4e2a-9d25-003749df8035 | Recipes: >
Execute:  <Order: id=534b628f-66d2-4248-8784-08d80a4e154b, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.17757892 BTC, size=372.09, price=2095.35, criteria=StopLoss: either | 0.02 (T/P) | 0.02 (S/L), path_id=34f6b9ce-6472-4e2a-9d25-003749df8035 | Recipes: >
Fill:  <Trade: id=8adf9107-2232-4cf4-9e42-380dd36d2d13, order_id=534b628f-66d2-4248-8784-08d80a4e154b, step=337, base_symbol=USD, quote_symbol=BTC, side=sell, type=market, quantity=370.97 USD, price=2041.98, commission=1.12 USD>
Total traded:  372.09
Completed:  534b628f-66d2-4248-8784-08d80a4e154b None
Execute:  <Order: id=3ce2c59d-3a12-4441-a131-83578bb4e834, status=0, type=market, side=sell, pair=USD/BTC, quantit

Completed:  25331f53-41d5-4d76-9c5f-3b8ae0318d87 None
Execute:  <Order: id=14d20fda-8491-4f4b-b3a2-86c475c243ea, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.43614299 BTC, size=918.6, price=2106.19, criteria=StopLoss: either | 0.01 (T/P) | 0.06 (S/L), path_id=06e72d08-dd38-4c76-9007-ff1821929c87 | Recipes: >
Fill:  <Trade: id=305edfa0-1770-48fb-8f62-e0cbd518178b, order_id=14d20fda-8491-4f4b-b3a2-86c475c243ea, step=348, base_symbol=USD, quote_symbol=BTC, side=sell, type=market, quantity=915.84 USD, price=1964.31, commission=2.76 USD>
Total traded:  918.6
Completed:  14d20fda-8491-4f4b-b3a2-86c475c243ea None
Execute:  <Order: id=90a8c0ae-f80b-441e-bf04-6a4979b8db7e, status=0, type=market, side=sell, pair=USD/BTC, quantity=1.03000078 BTC, size=2126.21, price=2064.28, criteria=StopLoss: either | 0.02 (T/P) | 0.04 (S/L), path_id=5ec80bc3-b9e4-42bc-9e7c-da9123def34d | Recipes: >
Fill:  <Trade: id=01f80294-68f0-4e48-bf11-15aa1ca19da3, order_id=90a8c0ae-f80b-441e-bf04-6a4979b8db7

Execute:  <Order: id=7f566504-736a-48c5-8477-3e29b613667f, status=0, type=market, side=buy, pair=USD/BTC, quantity=263.02 USD, size=263.02, price=1999.91, criteria=None, path_id=7f566504-736a-48c5-8477-3e29b613667f | Recipes: <Recipe: id=98d1a68e-3a8a-4ff9-b8dd-8851f0a88f3f, type=market, pair=USD/BTC, criteria=StopLoss: either | 0.02 (T/P) | 0.06 (S/L)>>
Fill:  <Trade: id=80d64fab-519e-42d9-8831-7d4623945363, order_id=7f566504-736a-48c5-8477-3e29b613667f, step=354, base_symbol=USD, quote_symbol=BTC, side=buy, type=market, quantity=262.23 USD, price=1999.91, commission=0.79 USD>
Total traded:  263.02
Completed:  7f566504-736a-48c5-8477-3e29b613667f <Order: id=0dd0fe28-5c2f-4981-afcd-c3655ca4d252, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.13112090 BTC, size=262.23, price=1999.91, criteria=StopLoss: either | 0.02 (T/P) | 0.06 (S/L), path_id=7f566504-736a-48c5-8477-3e29b613667f | Recipes: >
Execute:  <Order: id=f445c7b6-ebc3-44bf-9976-f6e5c970e7c0, status=0, type=market, s

Completed:  e3aa7347-b966-4a4c-82c1-2f46ae135573 None
Execute:  <Order: id=1e44595d-814f-4ad4-94e7-70f5d4289907, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.32171282 BTC, size=629.37, price=1956.31, criteria=StopLoss: either | 0.02 (T/P) | 0.04 (S/L), path_id=3d769d1f-0ee9-4590-ad03-df6628e407ce | Recipes: >
Fill:  <Trade: id=4cc265f1-484b-42b8-a8dc-0f5293c37934, order_id=1e44595d-814f-4ad4-94e7-70f5d4289907, step=360, base_symbol=USD, quote_symbol=BTC, side=sell, type=market, quantity=627.48 USD, price=1843.92, commission=1.89 USD>
Total traded:  629.37
Completed:  1e44595d-814f-4ad4-94e7-70f5d4289907 None
Execute:  <Order: id=a518c0b8-c65b-4f9d-9264-d8206111a806, status=0, type=market, side=sell, pair=USD/BTC, quantity=0.56697651 BTC, size=1113.78, price=1964.42, criteria=StopLoss: either | 0.03 (T/P) | 0.04 (S/L), path_id=6a30f2bb-6388-4579-a846-77cc281e74f5 | Recipes: >
Fill:  <Trade: id=a3fdcf70-ef53-4dbd-964e-7756205e9bb3, order_id=a518c0b8-c65b-4f9d-9264-d8206111a

In [6]:
%matplotlib inline

performance.net_worth.plot()

NameError: name 'performance' is not defined